## Generate Models

This code generates the models for a rectangular wing with a FWTD, it creates a series of models to test different non-linearaties

In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import odeint,LSODA,BDF,solve_ivp
from scipy.linalg import eig
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RectWing.Models as rwm
import multiprocessing as mp

me.mechanics_printing()

mp.cpu_count()

In [2]:
def Model_0():
    return rwm.Gen2DofModel(fwt_free=False,fwt_frot=False)
def Model_1():
    return rwm.Gen2DofModel(fwt_free=True,fwt_frot=False)
def Model_2():
    return rwm.Gen2DofModel(fwt_free=True,fwt_frot=True)
def Model_3():
    return rwm.Gen2DofModel(fwt_free=True,fwt_frot=True,rot_AoA=False)
def Model_4():
    model,p = rwm.Gen2DofModel(fwt_free=True,fwt_frot=True)
    return model.linearise(p).subs({i:0 for i in p.fp}),p
ModelFactory = {
    0:Model_0,
    1:Model_1,
    2:Model_2,
    3:Model_3,
    4:Model_4
}


def MakeModel(k,v):
    model,p = v()
    model.to_file(f'2DoF-M{k}.py')
    return k

In [7]:
pool = mp.Pool(mp.cpu_count())

for k,v in ModelFactory.items():
    pool.apply_async(MakeModel,args=(k,v))
pool.close()
pool.join()

In [3]:
p = rw.base_params(2)
sm_0 = mf.SymbolicModel.from_file('2DoF-M0.py')
sm_1 = mf.SymbolicModel.from_file('2DoF-M1.py')
sm_2 = mf.SymbolicModel.from_file('2DoF-M2.py')
sm_3 = mf.SymbolicModel.from_file('2DoF-M3.py')
sm_4 = mf.SymbolicModel.from_file('2DoF-M4.py')